# **Berlekamp-Massey Algorithm**

## Generator implementation

In [1]:
def print_poly(poly):
    poly = [i for i, b in enumerate(f'{poly:b}'[::-1]) if bool(int(b))]
    poly = ' + '.join([
        (f'x^{d}' if d > 1 else ('x' isf d==1 else '1')) 
        for d in poly[::-1]
    ])
    print(poly)

In [2]:
print_poly(0b1101)

x^3 + x^2 + 1


In [2]:
def berlekamp_massey(bits, debug=False):
    # bits -> b = [b0, b1, ... bN]
    b_len = len(bits)
    bits = int(''.join(f'{i:b}' for i in bits[::-1]), 2)
    
    P = 1
    m = 0
    Q = 1
    r = 1
    
    if debug:
        print(f' t | b | d | m |    P')
        print(f'-------------------------')
    
    for t in range(b_len):
        d = 0
        for j in range(m+1):
            d += ((P>>j) & 1) & ((bits>>(t-j)) & 1)
        d = d % 2
        
        if d == 1:
            if 2*m <= t:
                P, Q = P^(Q<<r), P
                
                m = t + 1 - m
                r = 0
            else:
                P = P^(Q<<r)
        
        if debug:
            print(f' {t} | {(bits>>t) & 1} | {d} | {m} |  {P:05b}')
        
        r += 1
        
    poly = [i for i, b in enumerate(f'{P:b}'[::-1]) if bool(int(b))]
    return poly

In [4]:
berlekamp_massey([1, 0, 1, 0, 0, 1, 1, 1], True)

 t | b | d | m |    P
-------------------------
 0 | 1 | 1 | 1 |  00011
 1 | 0 | 1 | 1 |  00001
 2 | 1 | 1 | 2 |  00101
 3 | 0 | 0 | 2 |  00101
 4 | 0 | 1 | 3 |  00001
 5 | 1 | 1 | 3 |  01011
 6 | 1 | 0 | 3 |  01011
 7 | 1 | 0 | 3 |  01011


[0, 1, 3]

## Streaming Implementation

In [18]:
class BerlekampMassey():
    def __init__(self):
        self._bits = 0b0
        
        self._P = 1
        self._m = 0
        self._Q = 1
        self._r = 1
        self._t = 0
    
    def __call__(self, bit):
        self._bits = (self._bits<<1) | int(bit)
        
        d = 0
        for j in range(self._m+1):
            d += ((self._P>>j) & 1) & ((self._bits>>(j)) & 1)
        d = d % 2
        
        if d == 1:
            if 2*self._m <= self._t:
                self._P, self._Q = self._P^(self._Q<<self._r), self._P
                
                self._m = self._t + 1 - self._m
                self._r = 0
                self._bits &= (1<<self._m) - 1
            else:
                self._P = self._P^(self._Q<<self._r)
        
        self._r += 1
        self._t += 1
        return self._P

In [8]:
berlekamp_massey = BerlekampMassey()

bit_stream = [1, 0, 1, 0, 0, 1, 1, 1]

for bit in bit_stream:
    poly = berlekamp_massey(bit)
    print(bin(poly))

0b11
0b1
0b1
0b1001
0b1001
0b1001
0b1
0b1


## Check algorithm with LFSR implementation

In [13]:
import sys, os
sys.path.append(os.path.abspath("../"))
from Cangini import LFSR

In [19]:
lfsr = LFSR([3, 1, 0])
berlekamp_massey = BerlekampMassey()

for _ in range(10):
    P = berlekamp_massey(int(next(lfsr)))

print_poly(P)

x^3 + x + 1


In [20]:
lfsr = LFSR([8, 5, 3, 1, 0])
berlekamp_massey = BerlekampMassey()

for _ in range(2**len(lfsr)):
    P = berlekamp_massey(int(next(lfsr)))

print_poly(P)

x^8 + x^5 + x^3 + x + 1


In [22]:
lfsr = LFSR([14, 13, 10, 8, 7, 5, 4, 3, 2, 1, 0])
berlekamp_massey = BerlekampMassey()

# Produce a number of bits double wrt the length of the LFSR
for _ in range(2*2**len(lfsr)):
    P = berlekamp_massey(int(next(lfsr)))

print_poly(P)

x^14 + x^13 + x^10 + x^8 + x^7 + x^5 + x^4 + x^3 + x^2 + x + 1
